## 0. Load libraries & environment variables

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_openai import ChatOpenAI

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

import logging
logging.getLogger("httpx").setLevel(logging.WARNING)

## 1. Retriever

In [82]:
########## IMPORTANT #############
# Set number of chunks for both the base retriever and reranker. 
# FlashRank reranker will NOT inherit it from base retriever. If not set gthe default top_n = 3 will be used
TOP_N_RETRIEVE = 10
TOP_N_RERANK = 5

### Base retriever

In [83]:
# access the stored vector store
embedding_model = HuggingFaceBgeEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
persist_directory = "vector_db_tenancy_agreements"
vectordb = Chroma(embedding_function=embedding_model,
                  persist_directory=persist_directory,
                  collection_name="tenecy_agreements")

# set as base retriever
retriever = vectordb.as_retriever(search_type = 'similarity', search_kwargs = {'k': TOP_N_RETRIEVE})

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


### Pre-retrieval query rewriting

In [85]:
def query_rewrite(query: str, llm: ChatOpenAI):
    query_rerite_prompt = f"""
    You are a helpful assistant that takes a user's query and turns it into a short sentence or paragraph 
    so that it can be used in a semantic similarity search on a vector database to return the most similar 
    chunks of content based on the rewriten query. It is very likely the user is going to ask for information 
    that is contained in a tenancy agreement. Make sure to extract the property address and tenant names if any is mentioned.
    Please make no comment, just return the rewritten query.

    query: {query}

    ai: """

    rewriten_query = llm.invoke(query_rerite_prompt)

    return rewriten_query

### Reranking

In [75]:
compressor = FlashrankRerank(top_n = TOP_N_RERANK, # make sure to set top_n explicitly as it won't inherit it from the base retriever
                            prefix_metadata='', # metadata (address and tenant names) are already in the text of each chunk so no need to inject here
                            score_threshold = 0.0 # lower the bar for quesitons such as "how many rental flats in Glasgow?"
                            ) 

compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever,
)

In [76]:
print(compression_retriever._calculate_keys)

<bound method BaseModel._calculate_keys of ContextualCompressionRetriever(base_compressor=FlashrankRerank(client=<flashrank.Ranker.Ranker object at 0x10ea05ed0>, top_n=4, score_threshold=0.0, model='ms-marco-MultiBERT-L-12', prefix_metadata=''), base_retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x28984cc50>, search_kwargs={'k': 7}))>


## 2. LLM

In [96]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0.3)

## 3. Prompt template

In [32]:
# Format the retrieved chunks
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [77]:
prompt_template = """Use the context provided to answer the user's question below. If you do not know the answer 
based on the context provided, tell the user that you do not know the answer to their question based on the context
provided and that you are sorry.

context: {context}

question: {query}

answer:
"""

custom_rag_prompt = PromptTemplate.from_template(prompt_template)

## 4. RAG Chain

In [97]:
class RAGChain:
    def __init__(
        self,
        llm: ChatOpenAI,
        retriever: ContextualCompressionRetriever,
        prompt: PromptTemplate
    ):
        self.llm = llm
        self.retriever = retriever
        self.prompt = prompt

    # method - invoke
    def invoke(self, query: str):
        # Pre-retrieval rewriting
        rewriten_query = query_rewrite(query, self.llm)
        
        # reranking
        docs = self.retriever.invoke(rewriten_query.content)
        
        # Formatting retrieved
        context = format_docs(docs)
        
        # Prompt Template
        final_prompt = self.prompt.format(context=context, query=query)
        
        # Invoke 
        return llm.invoke(final_prompt)

In [98]:
rag_chain = RAGChain(llm, compression_retriever, custom_rag_prompt)

In [136]:
def answer_query(query: str):
    return rag_chain.invoke(query).content

## 5. Test with questions

In [134]:
questions = ["Who are the tenants in 29 Marwick Street?",
             "What is the rent and the deposit of the flat on Maxwellton Street?",
             "Is there a guarantor for tenants in 22 Maxwellton Street?",
            "When did the tenancy start for flat at 166 Causewayside?",
             "What is the address of the rental flat in Edinburgh?",
             "What is the address of the rental property in G32?",
             "What is the general policy about keeping a pet in the rental flats?",
             "What are the responsibilities of the landlord if an emergency repair is needed?",
             "Is the tenant allowed to sublet a rental property?"
             "How to end a tenancy as a tenant?"
            ]

In [137]:
print("Testing Q&A: \n")
for query in questions:
    print("Q: " + query)
    print("A: " + answer_query(query) + "\n")

Testing Q&A: 

Q: Who are the tenants in 29 Marwick Street?
A: The tenants at 29 Marwick Street are Kyle Grant and Loreta Brunton.

Q: What is the rent and the deposit of the flat on Maxwellton Street?
A: The rent for the flat on Maxwellton Street is £545 a calendar month, and the deposit is £485.

Q: Is there a guarantor for tenants in 22 Maxwellton Street?
A: Yes, there is a guarantor for the tenants in 22 Maxwellton Street. The guarantor is Helen Govha.

Q: When did the tenancy start for flat at 166 Causewayside?
A: The tenancy for the flat at 166 Causewayside started on 16/05/2024.

Q: What is the address of the rental flat in Edinburgh?
A: The address of the rental flat in Edinburgh is Flat 7, 166 Causewayside, Edinburgh, EH9 1PJ.

Q: What is the address of the rental property in G32?
A: The address of the rental property in G32 is Flat 2/2, 4 Darleith Street, Glasgow, G32 7HZ.

Q: What is the general policy about keeping a pet in the rental flats?
A: The general policy about keep